In [18]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import numpy as np
import re

import seaborn as sns
from matplotlib             import pyplot as plt
from IPython.core.display   import HTML
from IPython.display        import Image

In [3]:
# define City
city = 'Berlin'

url = f'https://housinganywhere.com/s/{city}--Germany/apartment-for-rent'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get(url, headers=headers)

# instance BS
soup = BeautifulSoup(page.text, 'html.parser')

In [4]:
houses_list = soup.findAll('a', class_ = 'jss169 jss14')
houses_id = []

for h in range(24):
    houses_id.append([soup.findAll('a', class_ = 'jss169 jss14')[h].get('href').split('/')[2], soup.findAll('a', class_ = 'jss169 jss14')[h].get('href').split('/')[-1]])

In [5]:
# get all houses links from determinated city

# getting total pages number
a = soup.find('div', class_ = 'MuiGrid-root jss208 jss207 MuiGrid-container MuiGrid-justify-xs-flex-end')
b = a.findAll('a',class_ = 'MuiButtonBase-root MuiButton-root MuiButton-outlined jss209 MuiButton-outlinedSizeSmall MuiButton-sizeSmall')[3].span.text
b = int(b)

# Crete a empty listo for houses infos
houses_id = []

# going thru all pages from city
for page in range(b):
    url = f'https://housinganywhere.com/s/{city}--Germany/apartment-for-rent?page={page}'
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    houses_list = soup.findAll('a', class_ = 'jss169 jss14')
    
    # going thru all houses from page
    for h in range(len(houses_list)):
        id = soup.findAll('a', class_ = 'jss169 jss14')[h].get('href').split('/')[2]
        comp = soup.findAll('a', class_ = 'jss169 jss14')[h].get('href').split('/')[-1]
        
        url = f'https://housinganywhere.com/room/{id}/de/Berlin/{comp}'
        houses_id.append(url)
    
    houses_list.clear()
    
# len(houses_id)

In [20]:
df_houses = pd.DataFrame(columns = ['house_id', 'montly_rent', 'deposit_value', 'house_rules', 'street'])

houses_df_list = []

# get houses infos link by link

for house in houses_id:
    h = {}
    
    url = f'{str(house)}'

    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    page = requests.get(url, headers=headers)

    # Html 
    soup = BeautifulSoup(page.text, 'html.parser')

    # Hause ID
    try:
        h['house_id'] = url.split('/')[4]
    except:
        pass
    
    # Montly rent value
    try:
        h['montly_rent'] = soup.findAll('p', class_ = 'MuiTypography-root jss105 jss55 jss43 jss49 MuiTypography-h2 MuiTypography-displayBlock')[0].text
    except:
        pass
    
    # Deposit value
    try:
        if len(soup.findAll('div', class_ = 'MuiGrid-root MuiGrid-container MuiGrid-item MuiGrid-justify-xs-flex-end MuiGrid-grid-xs-6')) > 1:
            h['deposit_value'] = soup.findAll('div', class_ = 'MuiGrid-root MuiGrid-container MuiGrid-item MuiGrid-justify-xs-flex-end MuiGrid-grid-xs-6')[1].text
        else:
            h['deposit_value'] = soup.findAll('div', class_ = 'MuiGrid-root MuiGrid-container MuiGrid-item MuiGrid-justify-xs-flex-end MuiGrid-grid-xs-6')[0].text
    except:
        pass
    
    # Minimum stay
    # soup.findAll('div', class_ = 'MuiGrid-root MuiGrid-container MuiGrid-item MuiGrid-justify-xs-flex-end MuiGrid-grid-xs-6')[0].text

    # House rules - TRY TO FILTER "PET ALLOWED"
    try:
        h['house_rules'] = rules_list = []

        a =  soup.findAll('div', class_ = 'MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12')[1]

        b = a.findAll('div', class_ = 'MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12 MuiGrid-grid-md-4')

        for iten in range(len(b)):
            i = a.findAll('div', class_ = 'MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12 MuiGrid-grid-md-4')[iten].text
            rules_list.append(i)
    except:
        pass
    
    # Location (coordinates? Street?)
    try:
        a = soup.findAll('h1', class_ = 'MuiTypography-root jss52 jss43 jss46 MuiTypography-h1 MuiTypography-displayBlock')
        h['street'] = a[0].text
    except:
        pass
    
    # Get apartment size
    try:
        h['size'] = soup.findAll('p', class_ = 'MuiTypography-root jss59 jss43 jss46 MuiTypography-body1 MuiTypography-displayInline')[1].text
    except:
        pass
    
    # Get lat/long
    try:
        a = soup.findAll('script', type = 'application/ld+json')[0].get_text
        h['lat_long'] = re.findall('(\d+\.\d+)', str(a))
    except:
        pass
    
    h['scrapy_datetime'] = datetime.now().strftime('%Y/%m/%d - %H:%M:%S')
    
    
    
    houses_df_list.append(h)

In [21]:
df_houses = pd.DataFrame(houses_df_list)

In [22]:
df_houses.isna().sum()

house_id           0
montly_rent        5
deposit_value      0
house_rules        0
street             0
size               0
lat_long           0
scrapy_datetime    0
dtype: int64

In [23]:
df_houses.shape

(600, 8)

In [24]:
df_houses.head(10)

,house_id,montly_rent,deposit_value,house_rules,street,size,lat_long,scrapy_datetime
0,1719697,EUR 850,EUR 1700,"[Private living room, Private Toilet, Private ...",Nürnberger Straße,Property: 22 m2,"[52.50027, 13.33596]",2021/05/13 - 05:37:14
1,1724139,EUR 760,EUR 500,"[Private facilities, Private Toilet, Private b...",Otto-Braun-Straße,Property: 25 m2,"[52.52449, 13.41863]",2021/05/13 - 05:37:17
2,1721869,EUR 760,EUR 500,"[Private facilities, Private Toilet, Private b...",Otto-Braun-Straße,Property: 25 m2,"[52.52449, 13.41863]",2021/05/13 - 05:37:20
3,1719698,EUR 900,EUR 1800,"[Private living room, Private Toilet, Private ...",Nürnberger Straße,Property: 22 m2,"[52.50027, 13.33596]",2021/05/13 - 05:37:22
4,1721868,EUR 860,EUR 500,"[Private facilities, Private Toilet, Private b...",Otto-Braun-Straße,Property: 24 m2,"[52.52449, 13.41863]",2021/05/13 - 05:37:27
5,1208756,EUR 900,EUR 1800,"[Private living room, Shared garden, Private T...",Köpenicker Straße,Property: 26 m2,"[52.5097, 13.42231]",2021/05/13 - 05:37:29
6,1699607,EUR 990,EUR 0,"[Private facilities, Shared garden, Private To...",Takustraße,Property: 4000 m2,"[52.45695, 13.29344]",2021/05/13 - 05:37:32
7,1643513,EUR 950,EUR 1900,"[Private living room, Shared garden, Private T...",Köpenicker Straße,Property: 26 m2,"[52.5097, 13.42231]",2021/05/13 - 05:37:34
8,1717426,EUR 950,EUR 2000,"[Private living room, Private Toilet, Private ...",Koburger Straße,Property: 30 m2,"[52.4815, 13.34618]",2021/05/13 - 05:37:36
9,1208758,EUR 1200,EUR 2400,"[Private living room, Shared garden, Private T...",Köpenicker Straße,Property: 41 m2,"[52.5097, 13.42231]",2021/05/13 - 05:37:39


In [26]:
# save DF to CSV file
# df_houses.to_csv('data/df_houses_berlin_V1_with_junk')

In [ ]:
# get cities revenues